### 模式匹配  
定义对话模式，进行变量替换，即可以生成简单的对话：

```shell
”I want a ?X"  "Why do you want ?X" 
=>
”I want a Apple“ ”Why do you want Apple“
```


同时，一段对话中可能可以包含多个变量

基于这种原理，我们定义一种对话模式，当提问复合我们的对话模式时，我们可以通过模式匹配，得到回答。

输入：模式列表、一个问题  
输出：一个回答

问题处理过程：
1.  判断问题是否为对应的某个模式
2.  匹配到具体模式后，计算出对应的模式变量
3.  对模式的回答进行变量替换，输出回答


In [6]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [7]:
# saying 是否为对应模式，以及模式变量分别是什么
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [8]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [9]:
s = {"sss": "vvv"}

In [10]:
s.get("sss", "kkk")

'vvv'

In [11]:
s.get("ss", "kkk")

'kkk'

In [138]:
# 模式变量结构转换以及变量替换
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [13]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [14]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [15]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

#### 一个简单的使用

In [16]:
define_patterns = {
    "I will go to ?D": ["Why you go ?D", "Do you go to ?D to meet someone?"],
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"]
}

In [30]:
got_patterns = pat_match("I will go to ?D".split(), "I will go to school".split())

In [25]:
import random

In [47]:
# random response
' '.join(subsitite(random.choice(define_patterns["I will go to ?D"]).split(), pat_to_dict(got_patterns)))

'Why you go school'

## Problem 01 partI

In [69]:
def get_response(saying, rules):
    for k, v in rules.items():
        got_pattern = pat_match(k.split(), saying.split())
        if(got_pattern):
            return ' '.join(subsitite(random.choice(v).split(), pat_to_dict(got_pattern)))
    pass

In [70]:
get_response('I will go to ShangHai', define_patterns)

'Why you go ShangHai'

In [72]:
get_response('I need Apple', define_patterns)

'Image you will get Apple soon'

In [82]:
get_response('I want an Apple', define_patterns) # no response

### Segment match

从 `?X` 到 `?*X`，从而使得可以用 "?*X" 匹配 "an apple"

In [84]:
# 改写 is_variable 
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [85]:
is_pattern_segment("?*P")

True

In [88]:
is_pattern_segment("?P")

False

In [130]:
# 改写 pat_match
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat): # ?*P 因为 * 不是字母，这里无法满足
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying) # 这里有一个匹配算法
        if index == -1:
            return []
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return []

In [131]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    # return (seg_pat, saying), len(saying)
    # 匹配失败，返回 [] 0
    return _, -1

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]): # 如果遇到 rest 中的 * ？ 之类的，匹配完毕
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [135]:
segment_match("?*X hello ?*D".split(), "My dog is good")

(False, -1)

In [134]:
is_match(["hello"], ["yeast", "aa"])

False

In [133]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [91]:

response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [92]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [93]:
pat_match_with_seg('I need ?X'.split(), 
                  "I need an iPhone".split())

[('?X', 'an')]

In [116]:
pat_match_with_seg('?*x hello ?*y'.split(),"Yestady I was a student".split())

[('?x', ['Yestady', 'I', 'was', 'a', 'student'])]

In [94]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [95]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

In [180]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [98]:
Eng_rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']
}

## Problem 01 PartII

In [192]:
def get_response(saying, rules):
    for k, v in rules.items():
        got_pattern = pat_match_with_seg(k.split(), saying.split())
        if got_pattern:
            return ' '.join(subsitite(random.choice(v).split(), pat_to_dict(got_pattern)))
    
    pass

In [140]:
pat_to_dict([('?x', ['Yestady']), ('?y', ['a', 'student'])])

{'?x': ['Yestady'], '?y': ['a', 'student']}

In [144]:
subsitite("Perhaps I already knew you were ?y".split(), pat_to_dict([('?x', ['Yestady']), ('?y', ['a', 'student'])]))

['Perhaps', 'I', 'already', 'knew', 'you', 'were', ['a', 'student']]

In [208]:
get_response("Yestady I was a student", Eng_rule_responses)

'Perhaps I already knew you were a student'

#### 中文分词

In [207]:
pip install jieba

    100% |████████████████████████████████| 7.3MB 778kB/s ta 0:00:011
  Stored in directory: /Users/telnetning/Library/Caches/pip/wheels/c9/c7/63/a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


In [211]:
import jieba

In [213]:
zh_rule_responses = {
        '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [218]:
print("_".join(jieba.cut("我今天有点难过",cut_all=True)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/8b/_681s2t12zzfg84s_yp6lfm00000gn/T/jieba.cache
Loading model cost 1.379 seconds.
Prefix dict has been built succesfully.


我_今天_有点_难过


In [219]:
print("_".join(jieba.cut("我今天有点难过",cut_all=False))) # 也是默认模式

我_今天_有点_难过


In [220]:
print("_".join(jieba.cut("我来到北京清华大学",cut_all=True)))

我_来到_北京_清华_清华大学_华大_大学


In [221]:
print("_".join(jieba.cut("我来到北京清华大学",cut_all=False)))

我_来到_北京_清华大学


In [227]:
list(jieba.cut("我今天有点难过",cut_all=False))

['我', '今天', '有点', '难过']

### Problem 02

In [294]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    # seg_pat = seg_pat.replace('?*', '?')
    seg_pat = seg_pat[1:]
    
    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    # return (seg_pat, saying), len(saying)
    # 匹配失败，返回 _, 0
    return _, -1

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if rest[0][0].encode('UTF-8').isalpha(): # 如果遇到 rest 中的 字母 之类的，匹配完毕
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [346]:
def zh_sentence_to_list(sentence):
    return list(jieba.cut(sentence, cut_all=False, HMM=False))

In [230]:
zh_sentence_to_list("为什么有些人不喜欢运动呢")

['为什么', '有些', '人', '不', '喜欢', '运动', '呢']

In [254]:
zh_sentence_to_list("?*xhello有些人?*y")

['?', '*', 'xhello', '有些人', '?', '*', 'y']

In [ ]:
pat_match_with_seg("为什么有些人不喜欢运动呢","?*x有些人?*y")

In [225]:
def get_response(saying, rules):
    for k, v in rules.items():
        got_pattern = pat_match_with_seg(k.split(), saying.split())
        if got_pattern:
            return ' '.join(subsitite(random.choice(v).split(), pat_to_dict(got_pattern)))
    
    pass

In [257]:
import re

In [255]:
# jieba 默认对符号必定分割，为了将匹配符当做整体，需要先进行处理
# s = "?*x我们?b好的吗？?*M" => xxx我们bb好的吗？MMM
def wrapper_pattern(pattern):
    reg01 = '\?\*([a-zA-Z]+)'
    reg02 = '\?([a-zA-Z]+)'
    #re.search(reg01, s)
    pattern = re.sub(reg01, r'\1\1\1', pattern)
    pattern = re.sub(reg02, r'\1\1', pattern)
    return pattern

In [258]:
wrapper_pattern("?*x我们?b好的吗？?*M")

'xxx我们bb好的吗？MMM'

In [259]:
wrapper_pattern("?*x和?*y一样?*z")

'xxx和yyy一样zzz'

In [260]:
zh_sentence_to_list(wrapper_pattern("?*x和?*y一样?*z"))

['xxx', '和', 'yyy', '一样', 'zzz']

In [295]:
## 重新定义 is_variable 和 is_pattern_segment
def is_variable(pattern):
    return all(a.encode('UTF-8').isalpha() for a in pattern) and len(pattern) == 2

def is_pattern_segment(pattern):
    return all(a.encode('UTF-8').isalpha() for a in pattern) and len(pattern) == 3

In [322]:
# 改写 pat_match
# 递归匹配，当匹配失败时间，必须返回标识表示匹配失败，传递到最上层，这里标识为 None
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat): 
        subMatch = pat_match_with_seg(pattern[1:], saying[1:])
        if subMatch == None:
            return None
        return [(pat, saying[0])] + subMatch
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying) # 这里有一个匹配算法
        if index == -1:
            return None
        subMatch = pat_match_with_seg(pattern[1:], saying[index:])
        if subMatch == None:
            return None
        return [match] + subMatch
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return None

In [325]:
pat_match_with_seg(['xxx', '我', 'zzz', '梦见', 'yyy'], ['我', '和', '你', '一样', '都', '是', '学生', '吗'])

In [300]:
pat_match_with_seg(['xxx', '和', 'yyy', '一样', 'zzz'], ['我', '和', '你', '一样', '都是贫下中农'])

[('xx', ['我']), ('yy', ['你']), ('zz', ['都是贫下中农'])]

In [347]:
def get_response(saying, rules):
    for k, v in rules.items():
        got_pattern = pat_match_with_seg(zh_sentence_to_list(wrapper_pattern(k)), zh_sentence_to_list(saying))
        if got_pattern:
            return ''.join(subsitite(zh_sentence_to_list(wrapper_pattern(random.choice(v))), pat_to_dict(got_pattern)))
    
    return random.choice(rules.items[-1])

In [313]:
pat_match_with_seg(zh_sentence_to_list(wrapper_pattern("?*x我?*z梦见?*y")), zh_sentence_to_list("我和你一样都是学生吗"))

[('xx', [])]

In [314]:
zh_sentence_to_list(wrapper_pattern("?*x我?*z梦见?*y"))

['xxx', '我', 'zzz', '梦见', 'yyy']

In [315]:
zh_sentence_to_list("我和你一样都是学生吗")

['我', '和', '你', '一样', '都', '是', '学生', '吗']

In [320]:
pat_match_with_seg(['xxx', '我', 'zzz', '梦见', 'yyy'], ['我', '和', '你', '一样', '都', '是', '学生', '吗'])

[('xx', [])]

In [348]:
get_response('我和你一样都是学生吗', zh_rule_responses)

'是 学生 吗会对你有什么影响呢?'

In [349]:
get_response('我不要吃饭', zh_rule_responses)

'你说 不，是想表达不想的意思吗？'

In [350]:
get_response('关羽你为什么不来吃火锅', zh_rule_responses)

'你自己为什么不来 吃火锅'

In [351]:
get_response('哼哼哈吉快使用双截棍', zh_rule_responses) # 没有匹配到的情况

'我不太确定我很理解你说的, 能稍微详细解释一下吗?'

In [353]:
get_response('我要喝饮料', zh_rule_responses)

'我不太确定我很理解你说的, 能稍微详细解释一下吗?'

In [354]:
get_response('好的嗯嗯我马上来', zh_rule_responses)

'好的'

In [355]:
get_response('我和你都是贫下中农', zh_rule_responses)

'是 贫下中农会对你有什么影响呢?'

### problem 04

+  程序的优点是原理上简单，实现上也简单，利用极其有限的算法可以实现简单的算法；缺点是模式比较单一，需要针对每一个问题进行大量的匹配，并且很容易得到无法匹配，即无法理解问题的结果。要改进这个问题，一方面可以可以大量增加模式输入，但是这样可能会造成回答问题时效率的降低。另外，如果针对语句进行更深入的拆解，比如区分动词、名字、形容词，来帮助更好的区分解决问题，可能是一个方法。但是这个目前超出我能解决的范畴。


+   数据驱动，个人理解就是依据大量数据，得到或者提炼出某种规则，这种规则又可以反之解决其它的类似的问题。在该问题中，数据即提供的模式，它告诉了我们什么样的问题可以用什么样的模式解决，我们用工程实现这种模式，然后就可以反之用来回答类似的问题。  


+  数据驱动是 AI 的一种学习方式，通过大量的数据输入，经过各种拟合，可以得到类似于本例中的处理函数之类的规则，可以称之为模型，这种模型对于解决现实问题有巨大意义。AI 是一种学习的过程，学习的过程就是针对数据的学习，这就是数据驱动的意义。
